<a href="https://colab.research.google.com/github/tseykoroman/ML_for_business/blob/main/Tseyko_Lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
3. Вывести сравнение полученных моделей по метрикам бизнеса по показателям с урока
    - стоимость лечения 15000р, если сделали тест и начали лечить вовремя
    - стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы
    - стоимость теста 1400р
4. Сделать выводы о том, какая модель справилась с задачей лучше других
5. *Найти порог классификации по деньгам для лучшей модели
    - Стоимость лечения 15000р, если сделали тест и начали лечить вовремя
    - Стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы
    - Стоимость теста 1400р

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

In [2]:
df = pd.read_csv('C:/train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', axis=1), 
                                                    df['cardio'], random_state=0)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]




continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [8]:
def training(classifier):
    from sklearn.metrics import roc_auc_score

    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=7, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    # расчтаем метрики
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc_score = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
 
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score]]

In [9]:
results = pd.DataFrame([
    training(classifier=Pipeline([('features', feats), ('classifier', LogisticRegression(random_state=42))])),
    training(classifier=Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state=42))])),
    training(classifier=Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth=4, random_state=42))])),
], columns=['CV', 'Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])

results['models'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier']
results = results.set_index('models')

Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [10]:
results

,CV,Best Threshold,F,Precision,Recall,ROC AUC
models,,,,,,
LogisticRegression,0.786,0.387,0.730,0.647,0.838,0.784
RandomForestClassifier,0.774,0.350,0.719,0.643,0.816,0.771
DecisionTreeClassifier,0.787,0.380,0.731,0.647,0.840,0.789


Вывести сравнение полученных моделей по метрикам бизнеса по показателям с урока

стоимость лечения 15000р, если сделали тест и начали лечить вовремя

стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы

стоимость теста 1400р

In [11]:
import matplotlib.pyplot as plt
import itertools

In [14]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [17]:
def training(classifier):
    from sklearn.metrics import roc_auc_score

    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=7, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    # расчтаем метрики
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc_score = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
    business_threshold = 0

    rubl_ML_ = pow(10, 9)

    rs = []
    n = 20
    
    rubl_test = 1400
    rubl_early_treatment = 15000
    rubl_late_treatment = 20000
    
    
    for opt_buisness_tr in np.linspace(0, 1, n).tolist():
        # подберем порог для улучшения бизнесс показателя
        
        cnf_matrix = confusion_matrix(y_test, y_score > (opt_buisness_tr))
        TN = cnf_matrix[0][0]
        FN = cnf_matrix[1][0]
        TP = cnf_matrix[1][1]
        FP = cnf_matrix[0][1]

        rubl_1 = (TP + FN) * rubl_late_treatment
        rubl_test_all = np.sum(cnf_matrix) * rubl_test + (FN + TP) * rubl_early_treatment
        rubl_ML = (TP + FP) * rubl_test + FN * rubl_late_treatment + TP * rubl_early_treatment
        
        r_test_all_r_ML = rubl_test_all - rubl_ML 

        if rubl_ML < rubl_ML_:            
            business_threshold = opt_buisness_tr
            
            rubl_ML_ = rubl_ML
            
        rs.append(rubl_ML)  

            
    
#    print('расходы, если не делать тест и ждать симптомов:', rubl_1)
#    print('расходы, если делать тест всем и лечить группу больных:', rubl_test_all)
#    print('расходы, если использовать МЛ:', rubl_ML_)

#    print('Расходы "решение не делать тесты - расходы с МL:"', rubl_1 - rubl_ML_)
#    print('Расходы "решение не делать тесты - расходы сделать тесты всем":', rubl_1 - rubl_test_all)
#    print('Расходы "сделать тесты всем - решение делать тесты ML":', rubl_test_all - rubl_ML_)
    
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score, business_threshold]]

In [18]:
results = pd.DataFrame([
    training(classifier=Pipeline([('features', feats), ('classifier', LogisticRegression(random_state=42))])),
    training(classifier=Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state=42))])),
    training(classifier=Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth=4, random_state=42))])),
], columns=['CV', 'Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC', 'Уровень порога по бизнес метрике'])

results['models'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier']
results = results.set_index('models')

In [ ]:
results

,CV,Best Threshold,F,Precision,Recall,ROC AUC,Уровень порога по бизнес метрике
models,,,,,,,
LogisticRegression,0.786,0.387,0.730,0.647,0.838,0.784,0.316
RandomForestClassifier,0.774,0.350,0.719,0.643,0.816,0.771,0.211
DecisionTreeClassifier,0.787,0.380,0.731,0.647,0.840,0.789,0.263


Сделать выводы о том, какая модель справилась с задачей лучше других:


Если смотреть по roc auc, то лучше всего себя показывает дерево решений. Хотя в целом существенных отличий между деревом и лог. регрессией нет.

Если судить по бизнес метрике, то лучшую статистику показывает логистическая регрессия. 

Думаю, что если учитывать, что регрессия тоже имеет хорошие метрики по roc auc, то лучше всего подходит именно она.